<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/Pruebas/Graficar_BER_por_Bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Graficar BER por Bit

import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
import pandas as pd
from google.colab import files

plt.rcParams.update({'font.family': 'Arial'})



# Función para calcular intervalo de confianza
def confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    sem = stats.sem(data)  # Error estándar de la media
    h = sem * stats.t.ppf((1 + confidence) / 2, n - 1)  # Intervalo de confianza
    return mean, mean - h, mean + h


def test(qs):
  model.eval()
  torch.manual_seed(random.randint(0,1000))
  print('Qs', qs)
  num_train_block =  args.num_block
  errorxbit_T = torch.zeros(1, args.block_len, 1).to(device)
  promedio_errorxbit = torch.zeros(1, args.block_len, 1).to(device)
  BER_FINAL = []
  for q in qs:
    test_ber, test_bler = .0, .0
    with torch.no_grad():
      num_test_batch = int(num_train_block / (args.batch_size * test_ratio))
      for batch_idx in range(num_test_batch):
        if args.zero_padding:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          X_test = torch.cat([X_test, torch.zeros(args.batch_size, 1, 1)], dim=1)
          fwd_noise = torch.bernoulli(torch.full((args.batch_size, args.block_len + 1, args.code_rate), q))
          fb_noise = torch.zeros((args.batch_size, args.block_len + 1, args.code_rate))

        else:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), q))
          fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))

        X_test, fwd_noise, fb_noise = X_test.to(device), fwd_noise.to(device), fb_noise.to(device)
        X_hat_test, X_hat_test_code = model(X_test, fwd_noise, fb_noise)
        X_hat_test.to(device)
        X_hat_test_code.to(device)
        test_ber += errors_ber(X_hat_test, X_test)
        test_bler += errors_bler(X_hat_test, X_test)
        promedio_errorxbit += ber_pos(X_hat_test,X_test)

    promedio_errorxbit /= 1.0*num_test_batch
    errorxbit_T = torch.add(errorxbit_T, promedio_errorxbit)
    test_ber  /= 1.0*num_test_batch
    test_bler /= 1.0*num_test_batch

    print('Test Q',round(q,3) ,'with ber ', float(test_ber), 'with bler', float(test_bler))
    BER_FINAL.append(q)
    BER_FINAL.append(float(test_ber))

  return errorxbit_T, BER_FINAL

#PATH='torch_model_791480.pt'
#model=torch.load(PATH)

#q_interval = (args.q_test_end - args.q_test_start)* 1.0 /  (args.q_points)
#qs = [q_interval* item + args.q_test_start for item in range(args.q_points)]

qs=[0.01]

#history_errorxbit_Ta = torch.zeros(1, args.block_len, 1).to(device)
history_errorxbit_Ta = []


#for epoch in tqdm(range(1, args.num_epoch + 1)):
for epoch in tqdm(range(1, 11)):
    print(" ")
    print("Epoch: %d" % (epoch))
    print(" ")
    #train(epoch)
    errorxbit_Ta, BER_FINAL = test(qs)
    history_errorxbit_Ta.append(errorxbit_Ta.cpu().squeeze().numpy())


# Convertir history_errorxbit_Ta a numpy array para facilitar cálculos
history_errorxbit_Ta = np.array(history_errorxbit_Ta)

# Calcular el BER promedio y los intervalos de confianza
mean_ber_per_bit = np.mean(history_errorxbit_Ta, axis=0)

ci_low = []
ci_high = []

for i in range(args.block_len):
    _, low, high = confidence_interval(history_errorxbit_Ta[:, i])
    ci_low.append(low)
    ci_high.append(high)

ci_low=np.array(ci_low)
ci_high=np.array(ci_high)

lista = list(range(1, (args.block_len+1)))

# Crear DataFrame y guardar en Excel
bits = list(range(1, args.block_len + 1))
data = {
    'Bit': bits,
    'BER Promedio': mean_ber_per_bit,
    'Intervalo de Confianza Bajo': ci_low,
    'Intervalo de Confianza Alto': ci_high
}

df = pd.DataFrame(data)
df.to_excel('ber_por_bit_results.xlsx', index=False)

# Descargar el archivo Excel
files.download('ber_por_bit_results.xlsx')


Guardar_resultados(BER_FINAL, args, model)
#plt.plot(lista, lista_errxbit_q, marker='o', linestyle=':')
#plt.plot(lista, mean_ber_per_bit, marker='o', linestyle=':', color='blue')  # solo puntos azules
#plt.fill_between(lista, ci_low, ci_high, color=(1.0, 0.5, 0.5, 0.8), alpha=0.5, label='95% Confidence Interval')
#plt.errorbar(lista, mean_ber_per_bit, yerr=[mean_ber_per_bit - ci_low, ci_high - mean_ber_per_bit], fmt='o', linestyle='-', color='blue', ecolor='red', elinewidth=2, capsize=4, label='Mean BER with 95% CI')
plt.errorbar(lista, mean_ber_per_bit,
             yerr=[mean_ber_per_bit - ci_low, ci_high - mean_ber_per_bit],
             fmt='o', linestyle='-', color='blue',
             ecolor=(1.0, 0.5, 0.5, 0.8),  # Rojo suave con transparencia
             elinewidth=2, capsize=4,
             label='Mean BER with 95% CI')
plt.xlabel('Posición de bit')
plt.ylabel('BER')
plt.title('BER del bit bk')
plt.yscale('log')
plt.grid(True)  # Activa el grid
plt.ylim(1e-5, 1e0)  # Establece los límites del eje y de 10^0 a 10^-9
plt.savefig('grafico.png')  # Guarda la figura como un archivo PNG
plt.show()


files.download('grafico.png')
print("                                                                       ")
print("                                          ███████╗    ██╗    ███╗   ██╗")
print("                                          ██╔════╝    ██║    ████╗  ██║")
print("                                          █████╗      ██║    ██╔██╗ ██║")
print("                                          ██╔══╝      ██║    ██║╚██╗██║")
print("                                          ██║         ██║    ██║ ╚████║")
print("                                          ╚═╝         ╚═╝    ╚═╝  ╚═══╝")
print("                                                                       ")
print(model)
